In [33]:
import matplotlib.pyplot as plt
import numpy as np
from S2_read import *
from s2_and_sun_angs import *
S2file='/home/ardhuin/ADMIN/PROPOSALS/EE11/S2/DATA/T11SMS_20160429T183252';
#S2file='/home/datawork-WW3/IMAGERY/S2/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE/GRANULE/L1C_T11SMS_A004457_20160429T184037/IMG_DATA/T11SMS_20160429T183252'
boxi=[5600,6400,3600,4400] # bounding box indices in image:  East West South North 
bands=['B04','B03','B02'];  
nb=np.shape(bands)[0]
[imgs,NX,NY,nx,ny,dx,dy]=S2_read(S2file,boxi,bands)

# defines indices of where we are in coarse array of angles
# these indices are relative to top-left corner 
indx=int(np.round((0.5*(boxi[0]+boxi[1]))/(5000/dx)))
indy=int(np.round((NY*10/dx-0.5*(boxi[2]+boxi[3]))/(5000/dx)))
S2xml=S2file+'.xml'

#S2xml='/home/datawork-WW3/IMAGERY/S2/S2A_MSIL1C_20160429T183252_N0201_R027_T11SMS_20160429T184037.SAFE/GRANULE/L1C_T11SMS_A004457_20160429T184037/MTD_TL.xml'

(Tile_ID, AngleObs, AngleSun) = get_angleobs(S2xml)
ulx=AngleObs['ul_x']
uly=AngleObs['ul_y']
nz1=nz2=23
satg = numpy.zeros(shape=(nz1,nz2,13,3))
sung = numpy.zeros(shape=(nz1,nz2,5))
detector=numpy.zeros(shape=(nz1,nz2,13))

observe=AngleObs['obs']
sunall=AngleSun['sun']
sh1=np.shape(observe)
ncells=sh1[0]

for icell in range(0,ncells):
#  This takes the nearest cell in matrix 
   thiscell=observe[icell]
   iband=thiscell[0]
   ix=int((thiscell[2]-ulx)/5000)
   iy=int((uly-thiscell[3])/5000)
   detector[iy,ix,iband]=thiscell[1]
   satg[iy,ix,iband,0]=thiscell[6][0]
   satg[iy,ix,iband,1]=thiscell[6][1]
   satg[iy,ix,iband,2]=thiscell[6][2]

for isun in range(0,nz1*nz2):
   suncell=sunall[isun]
   ix=int((suncell[0]-ulx)/5000)
   iy=int((uly-suncell[1])/5000)
   sung[iy,ix,0]=suncell[2]
   sung[iy,ix,1]=suncell[3]
   sung[iy,ix,2]=suncell[4][0]
   sung[iy,ix,3]=suncell[4][1]
   sung[iy,ix,4]=suncell[4][2]


#observe = [bandId, detectorId, xcoord, ycoord, Sat, Gx]
#                        AngleObs['obs'].append(observe)


/home/ardhuin/ADMIN/PROPOSALS/EE11/S2/DATA/T11SMS_20160429T183252
6580
3600
(801, 801)
6580
3600
(801, 801)
6580
3600
(801, 801)


In [32]:
satg[indy,indx,2,:]

array([-0.10126618,  0.03955332,  0.99407278])

In [44]:
# these are the 3 components of the unit vector pointing to the sun
sun=sung[indy,indx,2:5]

thetav=np.zeros(shape=(nb,1))
phiv=thetav
offspec=thetav
phitrig=thetav
jj=0
for jb in ibands:
   # Unit vector from point in swath to sat
   sat=satg[indy,indx,jb,0:3]
   # vector that bisects the sun-target-sat angle 
   mid=sun+sat 
   midn=np.sqrt(mid[0]**2+mid[1]**2+mid[2]**2)
   mid=mid/midn;
   
   offspec[jj]=np.arccos(mid[2])*todeg            # off-specular angle
   phitrig[jj]=np.arctan2(mid[0],mid[1])*todeg   # azimuth of bistatic look
   jj=jj+1



In [40]:
np.shape(sat)

(3,)

In [21]:
sat[0,:]


array([ 0.3101452 , -0.3478892 ,  0.88475028])

In [17]:
offspec


array([138.28281452, 139.61621748, 139.70670128])